In [1]:
import numpy as np
from flask import Flask, request, jsonify
import pickle
import os
import pandas as pd
from scipy.sparse import hstack
from flask import render_template
from flask import redirect, url_for

In [2]:
os.getcwd()

'/Users/sauravjayan/Desktop/toxic-comment-classification'

In [3]:
app = Flask(__name__)
model = pickle.load(open('Saurav/lr_models.sav','rb'))

/Users/sauravjayan/opt/anaconda3/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.24.1 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [4]:
@app.route('/api',methods=['POST'])
def predict():
    data = request.get_json(force=True)
    prediction = model.predict([[np.array(data['exp'])]])
    output = prediction[0]
    return jsonify(output)

In [5]:
def requestResults(name):
    df = pd.DataFrame({'text':[name]})
    df = df['text']
#     print(df)
    word_vectorizer = pickle.load(open('Saurav/word_vectorizer.pkl','rb'))
    char_vectorizer = pickle.load(open('Saurav/char_vectorizer.pkl','rb'))
    test_word_features = word_vectorizer.transform(df)
    test_char_features = char_vectorizer.transform(df)
    test_features = hstack([test_char_features, test_word_features])
#     print(test_features)
    
    classifers = pickle.load(open('Saurav/lr_models.sav','rb'))
    y_test_toxic = classifers['toxic'].predict(test_features)
    y_test_severe_toxic = classifers['severe_toxic'].predict(test_features)
    y_test_obscene = classifers['obscene'].predict(test_features)
    y_test_threat = classifers['threat'].predict(test_features)
    y_test_insult = classifers['insult'].predict(test_features)
    y_test_identity_hate = classifers['identity_hate'].predict(test_features)
    return {'toxic':y_test_toxic,
           'sever_toxic': y_test_severe_toxic,
           'obscene':y_test_obscene,
           'threat':y_test_threat,
           'insult': y_test_insult,
           'identity_hate': y_test_identity_hate}
        

In [6]:
# start flask
app = Flask(__name__)

# render default webpage
@app.route('/')
def home():
    return render_template('home.html')

# when the post method detect, then redirect to success function
@app.route('/', methods=['POST', 'GET'])
def get_data():
    if request.method == 'POST':
        user = request.form['search']
        return redirect(url_for('success', name=user))

# get the data for the requested query
@app.route('/success/<name>')
def success(name):
    results = []

    d = requestResults(name)
    
    for key,value in d.items():
        if value[0] == 1:
            results.append(key)
            
    for i,label in enumerate(results):
        if label =='sever_toxic':
            results[i] = 'severely toxic'
        if label =='threat':
            results[i] = 'threatening' 
        if label =='insult':
            results[i] = 'insulting'
        if label =='identity_hate':
            results[i] = 'related to identity hate' 
        
            
    label_str = ''
    clean = True
    if len(results) != 0:
        for i,label in enumerate(results):
            if label != 'clean':
                if i != (len(results)-1):
                    label_str = label_str + ' ' + str(label) + ','
                else:
                    label_str = label_str + " " + 'and ' + str(label) + '.'
                clean=False
    else:
        label_str = ' clean.'
    
    if not clean:
        path='/images/giphy.gif'    
        image_file = url_for('static', filename=path)
    else:
        path='/images/goodjob.gif'    
        image_file = url_for('static', filename=path)
            
    return render_template('results.html',label=label_str, clean=clean, pic=image_file)

In [7]:
app.run(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [12/Mar/2021 11:44:20] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Mar/2021 11:44:32] "POST / HTTP/1.1" 302 -
/Users/sauravjayan/opt/anaconda3/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.24.1 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/sauravjayan/opt/anaconda3/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.24.1 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/sauravjayan/opt/anaconda3/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.24.1 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.